# 3/4- Clean&filter

**Rappel de la définition de l'objet d'étude** : "*Les paroles politiques (ayant un objet proprement politique) portant sur l'idée de "République" de personnels politiques (députés et ministres) dans le cadre de séance publique à l'Assemblée nationale.*"

--> Ce notebook vise à effectuer un premier nettoyage dans les données utilisées (Compte-rendus de séance de l'Assemblée nationale) pour ne conserver que les paroles politiques de personnels politiques en séance publique (sur les points de vigileance et spécificités de ce corpus voir le document annexe Docs/Données_Débats_AN.md). 

==> Cela signifie que l'on ne s'intéresse pas aux : 
1. paroles formelles relatives au fonctionnement/réglement de l'AN (introduction de la séance, résultat des votes, annonce de mise au vote, etc.)
2. para-texte (les didascalies contextuelles rédigées par le bureau de l'AN)
3. paroles de personnels non-politiques extérieurs. 

Ce nettoyage est effectué en amont de la définition des prises de paroles sur l'idée de "République" pour pouvoir effectuer une comparaison et mettre ces prises de paroles en % des prises de paroles totales (de leurs auteurs et de l'AN). 

## To-do : 

1. Définir ce qu'on fait des interruptions / Définir quelle est l'unité topique d'analyse
2. Mettre mention gouvernement et compléter manuellement affiliation politique

## Exclure les paroles non proprement politiques

La première option envisagée avait été d'utiliser la variable "code_grammaire" et ne retenir que les modalités "PAROLE_GÉNÉRIQUE" et "INTERRUPTION" mais en réalité il existe trop de modalités supplémentaires comprenant à la fois une parole politique ("Parole_1_2") et une parole formelle ("Parole_1_2") pour qu'il soit pertinent de faire une liste d'exclusion ou d'inclusion. (pour des explications plus avancées cf Docs/Données_Débats_AN.md)

Impossible aussi d'utiliser la variable "code_parole" et sa modalité "Parole_1_2" car 54% de valeurs manquantes. 

==> Il serait préférable de distinguer les paroles par la fonction des personnes qui les prononce et en l'occurence principalement la présidence de l'Assemblée Nationale ou directement le service des compte-rendus (pour des informations sur les types de paroles de la présidence cf Docs/Données_Débats_AN.md et src/Notebook/Annexe_Exclusions_AN.ipynb).

==> 
1. Ne conserver dans la colonne/variable "code_style" que la modalité "NORMAL" afin d'exclure le para-texte écrit exclusivement par le  *service des compte-rendus*
2. Supprimer les didascalies à l'intérieur des prises de paroles 
3. Supprimer toutes les prises de paroles de la présidence car celles-ci ont une nature en grande majorité (voir comment quantifier) formelles (cf src/Notebook/Annexe_Exclusions_AN.ipynb).


In [16]:
import pandas as pd

df = pd.read_csv(
   "/Users/matthiaslevalet/Desktop/Projet de recherche/CSS_République/Data/Interim/Data_AN_CSS.csv" , low_memory=False, dtype={"ID_orateur": str}
)

df.shape

(1128128, 25)

### Étape 1 : Ne garder que le texte normal

Dans la **variable "code_style"**, il existe 4 modalités : 
- "NORMAL" qui correspond à des prises de paroles (91%)
- "Infos Italiques" qui correspondent à des didascalies hors du texte, rajouté avant ou après les interventions pour donner le contexte par le service du compte-rendu (9%)
- "Signature droite" en fin de chaque CRI qui correspond à un texte comme "Le Directeur du service du compte rendu de la séance de l’Assemblée nationale" ou "La directrice" avec son nom (- 1%)
- D'autres " (-1%)
            
==> Il faut donc commencer par ne garder que la modalité "NORMAL". Ce qui nous fait passer de 1128128 à 1027459 soit une réduction de 100 669 dont 98800 de "Infos italiques" et 1844 de "Signatures droites".  

(Pour une observation plus détaillée voir Annexe_Exclusions_AN.ipynb)

In [ ]:
df = df[df["code_style"] == "NORMAL"]



### Étape 2 : Supprimer les didascalies dans les prises de paroles 

Elles ne sont plus en italiques mais sont comprises entre parenthèses donc utiliser une regex d'exclusion à l'intérieur de la colonne texte. On en profite pour introduire un nettoyage plus général pour enlever les balises html et espaces multiples

In [18]:
import re 

def nettoyer_texte(texte):
    if not isinstance(texte, str):
        return texte
    # Supprimer les balises HTML/XML
    texte = re.sub(r"<[^>]+>", "", texte)
    # Supprimer contenu entre parenthèses
    texte = re.sub(r"\([^)]*\)", "", texte)
    # # TODO: aviser si enlève crochets, ou juste […] [...], surtout des troncatures citations
    # texte = re.sub(r"\[[^\]]*\]", "", texte)
    # Supprimer les espaces multiples
    texte = re.sub(r"\s+", " ", texte).strip()
    return texte


df["texte_propre"] = df["texte"].apply(nettoyer_texte)

df = df.drop(columns=["texte"]) #la colonne du texte avant process n'est pas nécessaire (sera gardé dans le précédent fichier) et pèse lourd sur le fichier (+ de 300mo)

### Étape 3 : Supprimer les interventions de la présidence de l'AN 

(Risques en termes de pertes de paroles politiques minimes et essentiellement liés à la fonction de "représentation de l'assemblée nationale" et donc de son expression face à certains événements d’une particulière gravité)
**À voir si pertinent de garder les interventions dépassant un certain nombre de caractère à cet égard...**

Pour une observation des interventions spécifiques de la présidence de l'AN et de leurs caractéristiques, cf src/notebook/Annexe_Interventions_PR_AN.ipynb

On passe de 1 027 459 à 683 680 soit près de 343 799 de moins !! On perd aussi toutes les "Parole_1_1" donc pas besoin de ne garder que les paroles génériques et interruptions !!

Pour cela on utilise une fonction qui ne prend dans le DF que les occurences n'ayant pas M. le président ou MMe la présidente dans la colonne nom-orateur. Il s'agit donc d'une fonction boolèene inversée. 

In [19]:
# Exclure les prises de parole de "Mme la présidente" et "M. le président"
# Role_debat n'est pas bien identifié, utiliser Nom_orateur
df = df[~df["nom_orateur"].str.strip().isin(["M. le président", "Mme la présidente"])] # ici le ~ sert à enlever les occurences de ce qui est défini après donc pour étudier en détail les interventions de la présidence il faut juste l'enlver

df.shape

(683680, 25)

In [13]:
df.groupby("code_grammaire", dropna=False)["code_parole"].describe()

,count,unique,top,freq
code_grammaire,,,,
49_3_2_1,27,1,PAROLE_1_2,27
ACCORD_CONV_2_1,0,0,NaN,NaN
ADOP_ADT,0,0,NaN,NaN
ADOP_ADTS,0,0,NaN,NaN
ADT_SORTS_DIRECT_1_20,0,0,NaN,NaN
...,...,...,...,...
VOTE_ENS_PPR_S_2_30,0,0,NaN,NaN
VOTE_ENS_PPR_S_2_50,0,0,NaN,NaN
VOTE_ENS_PPR_S_2_80,0,0,NaN,NaN


In [20]:
# Autres étapes potentielles !

# Garder une trace de la longueur des interventions brutes
df["len_dirtytext"] = df["texte_propre"].str.len()

# Changer les missing values pour non_précisé (majoritaire) dans Code_parole
df["code_parole"] = df["code_parole"].fillna("non_précisé")

df.shape

(683680, 26)

In [7]:
# aperçu des répartitions
df.groupby("code_parole", dropna=False)["len_dirtytext"].describe()

,count,mean,std,min,25%,50%,75%,max
code_parole,,,,,,,,
(null),1.0,186.000000,NaN,186.0,186.00,186.0,186.00,186.0
AVIS_COM_1_10,1.0,1278.000000,NaN,1278.0,1278.00,1278.0,1278.00,1278.0
AVIS_COM_1_20,45107.0,450.390582,482.536522,3.0,118.50,315.0,618.00,8179.0
AVIS_GVT_1_20,38454.0,486.647371,681.527170,4.0,17.00,240.0,685.00,11340.0
PAROLE_1_1,21.0,556.761905,1080.603346,30.0,101.00,241.0,529.00,5088.0
PAROLE_1_2,247778.0,974.519219,1294.330875,3.0,245.00,547.0,1207.00,68092.0
PRESIDE_DISCOURS_1_10,2.0,95.500000,92.630988,30.0,62.75,95.5,128.25,161.0
Raccroche_apres_inter,9.0,1638.666667,1450.025431,60.0,154.00,1340.0,2316.00,3939.0
non_précisé,352287.0,282.342874,625.719777,1.0,17.00,37.0,251.00,25609.0


## Exporter les données 

In [21]:
import csv  # pour résoudre le soucis d'écart. Checker

df.to_csv(
    "/Users/matthiaslevalet/Desktop/Projet de recherche/CSS_République/Data/Interim/Data_AN_CSS_clean.csv",
    index=False,
    quoting=csv.QUOTE_ALL,  # a permis de résoudre le soucis d'écart. Checker
)